In [1]:
import os
import json
import pandas as pd
import datetime
from youtube_api import YoutubeDataApi

key = os.environ.get('YT_KEY')
yt = YoutubeDataApi(key)

In [2]:
def dump(doc):
    def default_handler(o):
        if isinstance(o, datetime.datetime):
            return o.isoformat()
    
    print(json.dumps(doc, sort_keys=True, indent=4, default=default_handler))

## A workflow sample

You may start with a channel name like 'LastWeekTonight' or 'TheNewYorkTimes'. Any data collected about channels must be collected using the channel ID, not the channel name.

In [3]:
channel_id = yt.get_channel_id_from_user('LastWeekTonight')
channel_id

'UC3XTzVzaHQEd30rQbuvCtTQ'

In [4]:
channel_meta = yt.get_channel_metadata(channel_id)
dump(channel_meta)

{
    "account_creation_date": "2014-03-18T17:41:39",
    "channel_id": "UC3XTzVzaHQEd30rQbuvCtTQ",
    "collection_date": "2019-04-09T17:56:52.350039",
    "country": null,
    "description": "Breaking news on a weekly basis. Sundays at 11PM - only on HBO.\nSubscribe to the Last Week Tonight channel for the latest videos from John Oliver and the LWT team.",
    "keywords": null,
    "playlist_id_likes": "LL3XTzVzaHQEd30rQbuvCtTQ",
    "playlist_id_uploads": "UU3XTzVzaHQEd30rQbuvCtTQ",
    "subscription_count": "6919153",
    "title": "LastWeekTonight",
    "topic_ids": "https://en.wikipedia.org/wiki/Humour|https://en.wikipedia.org/wiki/Entertainment|https://en.wikipedia.org/wiki/Television_program",
    "video_count": "267",
    "view_count": "1933056961"
}


In [5]:
subscriptions = yt.get_subscriptions(channel_id)
dump(subscriptions[:2])

[
    {
        "collection_date": "2019-04-09T17:56:54.862536",
        "subscription_channel_id": "UCWPQB43yGKEum3eW0P9N_nQ",
        "subscription_kind": "youtube#channel",
        "subscription_publish_date": "2014-03-20T19:05:54",
        "subscription_title": "HBOBoxing"
    },
    {
        "collection_date": "2019-04-09T17:56:54.862536",
        "subscription_channel_id": "UCy6kyFxaMqGtpE3pQTflK8A",
        "subscription_kind": "youtube#channel",
        "subscription_publish_date": "2014-12-11T18:55:41",
        "subscription_title": "Real Time with Bill Maher"
    }
]


## Getting uploads by a user
YouTube is consructed such that the video uploads by a user are stored in a playlist based on the user's channel ID. We can use this detail to generate the Upload Playlist ID for a given user and collect all videos posted by them.

In [6]:
from youtube_api import youtube_api_utils as utils

playlist_id = utils.get_upload_playlist_id(channel_id)
playlist_id

'UU3XTzVzaHQEd30rQbuvCtTQ'

### Using this, we can now pull the uploads for this `playlist_id`

In [8]:
videos = yt.get_videos_from_playlist_id(playlist_id)
df = pd.DataFrame(videos[:5])
df

,publish_date,video_id,channel_id,collection_date
0,2019-04-08 06:30:00,jCC8fPQOaxU,UC3XTzVzaHQEd30rQbuvCtTQ,2019-04-09 17:58:41.010311
1,2019-04-01 06:30:01,m8UQ4O7UiDs,UC3XTzVzaHQEd30rQbuvCtTQ,2019-04-09 17:58:41.011306
2,2019-03-18 06:30:01,Yq7Eh6JTKIg,UC3XTzVzaHQEd30rQbuvCtTQ,2019-04-09 17:58:41.011306
3,2019-03-11 06:30:00,FO0iG_P0P6M,UC3XTzVzaHQEd30rQbuvCtTQ,2019-04-09 17:58:41.011714
4,2019-03-04 07:30:01,_h1ooyyFkF0,UC3XTzVzaHQEd30rQbuvCtTQ,2019-04-09 17:58:41.011714


In [12]:
video_meta = yt.get_video_metadata(df.video_id.tolist())
dump(video_meta[:1])

[
    {
        "channel_id": "UC3XTzVzaHQEd30rQbuvCtTQ",
        "channel_title": "LastWeekTonight",
        "collection_date": "2019-04-09T18:00:17.342944",
        "video_category": "24",
        "video_comment_count": "11400",
        "video_description": "Mobile homes may seem like an affordable housing option, but large investment companies are making them less and less so.\n\nConnect with Last Week Tonight online... \n\nSubscribe to the Last Week Tonight YouTube channel for more almost news as it almost happens: www.youtube.com/lastweektonight \n\nFind Last Week Tonight on Facebook like your mom would: www.facebook.com/lastweektonight \n\nFollow us on Twitter for news about jokes and jokes about news: www.twitter.com/lastweektonight \n\nVisit our official site for all that other stuff at once: www.hbo.com/lastweektonight",
        "video_dislike_count": "2618",
        "video_id": "jCC8fPQOaxU",
        "video_like_count": "95580",
        "video_publish_date": "2019-04-08T06:30

In [10]:
searches = yt.search('john oliver', max_results=5)
dump(searches[:2])

[
    {
        "channel_id": "UC3XTzVzaHQEd30rQbuvCtTQ",
        "channel_title": "LastWeekTonight",
        "collection_date": "2019-04-09T18:00:03.466442",
        "video_category": null,
        "video_description": "John Oliver discusses how the WWE takes care of its wrestlers \u2014 and how it doesn't. Connect with Last Week Tonight online... Subscribe to the Last Week ...",
        "video_id": "m8UQ4O7UiDs",
        "video_publish_date": "2019-04-01T06:30:01",
        "video_thumbnail": "https://i.ytimg.com/vi/m8UQ4O7UiDs/hqdefault.jpg",
        "video_title": "WWE: Last Week Tonight with John Oliver (HBO)"
    },
    {
        "channel_id": "UC3XTzVzaHQEd30rQbuvCtTQ",
        "channel_title": "LastWeekTonight",
        "collection_date": "2019-04-09T18:00:03.466442",
        "video_category": null,
        "video_description": "Mobile homes may seem like an affordable housing option, but large investment companies are making them less and less so. Connect with Last Week Tonight

In [15]:
recommendations = yt.get_recommended_videos('m8UQ4O7UiDs')
dump(recommendations[:2])

[
    {
        "channel_id": "UC3XTzVzaHQEd30rQbuvCtTQ",
        "channel_title": "LastWeekTonight",
        "collection_date": "2019-04-09T18:02:15.601936",
        "video_category": null,
        "video_description": "Chinese president Xi Jinping is amassing an alarming amount of political power. If only his propaganda videos made the idea of unrestricted authority seem as troubling as the concept of singing children.\n\nConnect with Last Week Tonight online...\n \nSubscribe to the Last Week Tonight YouTube channel for more almost news as it almost happens: www.youtube.com/user/LastWeekTonight\n \nFind Last Week Tonight on Facebook like your mom would: http://Facebook.com/LastWeekTonight\n \nFollow us on Twitter for news about jokes and jokes about news: http://Twitter.com/LastWeekTonight\n \nVisit our official site for all that other stuff at once: http://www.hbo.com/lastweektonight",
        "video_id": "OubM8bD9kck",
        "video_publish_date": "2018-06-18T01:16:03",
        "v